In [1]:
import torch
import torch.nn as nn

Батч нормализация работает вдоль канала, то есть если у нас тензор размера ЧИСЛО_КАРТИНОК, ЧИСЛО_КАНАЛОВ, ВЫСОТА, ШИРИНА, то считаем среднее и дисперсию для каждого канала (То есть их размер будет 1, ЧИСЛО_КАНАЛОВ, 1, 1). Усредняем по всему батчу каждый канал Нормализация по слою работает по другому, она нормализует по любому количеству последних размерностей. То есть если у нас текст и нам надо нормализовать эмбеддинги, то нормализуем по их размерности. Вход будет ЧИСЛО_ТЕКСТОВ, ДЛИНА_ТЕКСТА, РАЗМЕР_ЭМБЕДДИНГА, то посчитав среднее и дисперсию получим (ЧИСЛО_ТЕКСТОВ, ДЛИНА_ТЕКСТА, 1), то есть среднее и дисперсию для каждого токена каждого текста в батче, считаем только по последней размерности.
Но нам ничего не мешает взять и посчитать по другому количеству последних осей, например по двум, то есть среднее будет посчитано как (ЧИСЛО ТЕКСТОВ, 1, 1), то есть усредняем каждый текст

In [2]:
# batchnorm
batch_size, channels_num, w, h = 32, 3, 15, 12

input = torch.randn(batch_size, channels_num, w, h)
batchnorm = nn.BatchNorm2d(channels_num)

print('Input:', input)

output_batchnorm = batchnorm(input)
print('Output torch:', output_batchnorm)

mean = torch.mean(input, dim=(0, 2, 3), keepdims=True)
print('Mean shape:', mean.shape)
eps = 1e-5
std = torch.std(input+eps, dim=(0, 2, 3), keepdims=True, correction=0)
print('Std shape:', std.shape)

y_custom = (input - mean) / (std)
print("y_custom:", y_custom)
assert torch.allclose(output_batchnorm, y_custom), 'Tensors do not match.'

Input: tensor([[[[ 1.4216e+00,  8.0401e-02,  9.0978e-02,  ...,  1.9635e+00,
            1.7225e-02,  4.9683e-01],
          [ 6.6707e-01, -7.5728e-01, -7.3061e-01,  ..., -8.6992e-01,
           -1.2541e+00, -2.8924e-01],
          [ 1.9928e-01, -5.1717e-01, -1.6158e-01,  ...,  1.3696e+00,
           -1.9239e+00,  1.7282e+00],
          ...,
          [ 1.1153e+00, -1.4951e+00,  9.1393e-01,  ...,  2.5453e-01,
           -9.3157e-01,  8.7169e-01],
          [-3.8310e-01, -1.6926e-01,  6.2048e-01,  ..., -1.0601e+00,
            1.7872e-01, -4.7018e-02],
          [ 1.8067e+00, -5.6509e-01, -8.3066e-02,  ...,  2.0884e+00,
           -2.3519e-01,  3.6769e-01]],

         [[ 1.3404e+00, -5.7249e-01,  1.2802e-01,  ..., -4.9320e-01,
           -1.5575e+00,  2.6315e-01],
          [-1.1424e+00,  2.6035e-01, -1.9849e-01,  ...,  3.6704e-01,
           -1.0501e+00,  2.9314e-01],
          [ 5.1021e-01, -1.4923e+00, -2.6227e-01,  ..., -8.4277e-01,
           -5.8500e-01,  8.9803e-01],
          ...

In [3]:
# batchnorm1d, тоже самое, по каналам
batch_size, channels_num, w = 32, 3, 15

input = torch.randn(batch_size, channels_num, w)
batchnorm = nn.BatchNorm1d(channels_num)

print('Input:', input)

output_batchnorm = batchnorm(input)
print('Output torch:', output_batchnorm)

mean = torch.mean(input, dim=(0, 2), keepdims=True)
print('Mean shape:', mean.shape)
eps = 1e-5
std = torch.std(input+eps, dim=(0, 2), keepdims=True, correction=0)
print('Std shape:', std.shape)


y_custom = (input - mean) / (std)
print("y_custom:", y_custom)
assert torch.allclose(output_batchnorm, y_custom), 'Tensors do not match.'

Input: tensor([[[ 0.7287,  0.1174, -0.5000,  ...,  0.1803,  0.2673,  0.5513],
         [ 0.2676,  1.8641, -0.6113,  ...,  0.0365,  0.1494,  0.1494],
         [-1.0503, -0.5056, -0.7208,  ...,  0.0037, -0.4637, -0.8866]],

        [[ 0.5234, -0.8975, -0.0729,  ...,  0.9338,  0.7250,  0.9426],
         [ 0.6881,  0.8599, -0.4228,  ...,  0.7508, -0.4055, -1.5475],
         [ 1.8842, -1.0397, -1.8769,  ..., -0.5834, -0.5092, -1.3901]],

        [[ 0.6656, -0.3471, -1.1018,  ...,  0.1862,  0.9948, -0.2536],
         [ 0.3142, -1.7226, -0.9208,  ...,  0.6874, -0.6144,  0.0546],
         [ 1.7967,  0.7607, -0.2701,  ...,  0.7545, -0.4785, -0.0413]],

        ...,

        [[-0.0046,  0.1576, -0.0309,  ..., -0.1643,  0.4164, -1.0802],
         [-0.1160,  0.3209, -0.9904,  ..., -0.0072,  0.1372,  1.7149],
         [ 1.9745, -0.0291,  0.6145,  ..., -0.0212,  0.9842, -1.5368]],

        [[ 1.6237,  2.3903,  0.9654,  ..., -1.5223,  0.3793,  1.4143],
         [-1.1656,  0.4037,  0.4997,  ...,  0.58

In [6]:
# layernorm
batch_size, seq_len, emb_size = 32, 30, 16

input = torch.randn(batch_size, seq_len, emb_size)
layernorm = nn.LayerNorm(emb_size, elementwise_affine=False, bias=False)

print('Input:', input)

output_layernorm = layernorm(input)
print('Output torch:', output_layernorm)

mean = torch.mean(input, dim=(2), keepdims=True) # считаем среднее по каждому элементу батча и каждому "токену"
print('Mean shape:', mean.shape)

eps = 1e-5
var = torch.var(input, dim=2, keepdims=True, correction=0)
print('Var shape:', std.shape)

y_custom = (input - mean) / torch.sqrt(var+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_layernorm, y_custom, atol=1e-6), 'Tensors do not match.'

Input: tensor([[[-2.3915e-01, -1.3354e+00,  1.0735e-01,  ...,  6.3129e-01,
          -1.0473e+00, -7.6756e-01],
         [ 1.4652e+00,  3.0020e-01, -4.7167e-04,  ...,  2.5564e-01,
           1.7531e+00,  3.3967e-01],
         [ 6.9079e-01, -4.0711e-01,  1.8903e-01,  ...,  1.4683e+00,
           8.6810e-01,  1.2444e+00],
         ...,
         [ 1.2381e+00,  1.6377e+00, -2.2812e-01,  ..., -4.8690e-01,
           1.4824e+00, -2.1230e+00],
         [-6.4828e-01, -7.2458e-02,  1.7165e-01,  ...,  3.0555e+00,
           3.6634e-01, -1.5151e-01],
         [-1.0555e+00, -1.0501e+00, -5.4684e-01,  ..., -2.1134e-01,
          -6.1648e-01, -9.5341e-01]],

        [[ 1.6718e-01,  3.7407e-02, -1.5047e+00,  ...,  5.0956e-02,
           4.6067e-01,  1.0355e+00],
         [-2.4705e-01, -2.7431e+00,  1.7571e+00,  ..., -6.3320e-01,
          -3.4068e-01,  5.1021e-01],
         [-9.3045e-01, -5.9135e-01,  1.0325e+00,  ..., -8.0165e-03,
          -2.7539e-01,  1.6260e+00],
         ...,
         [ 3.9978e

In [5]:
# layernorm, можно взять произвольное количество последних осей
batch_size, seq_len, emb_size = 32, 30, 16

input = torch.randn(batch_size, seq_len, emb_size)
layernorm = nn.LayerNorm((seq_len, emb_size), elementwise_affine=False, bias=False)

print('Input:', input)

output_layernorm = layernorm(input)
print('Output torch:', output_layernorm)

mean = torch.mean(input, dim=(1, 2), keepdims=True) # считаем среднее по каждому элементу батча и каждому "токену"
print('Mean shape:', mean.shape)

eps = 1e-5
var = torch.var(input, dim=(1, 2), keepdims=True, correction=0)
print('Var shape:', std.shape)

y_custom = (input - mean) / torch.sqrt(var+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_layernorm, y_custom, atol=1e-7), 'Tensors do not match.'

Input: tensor([[[ 0.4058,  0.3790,  0.3159,  ..., -0.4262,  2.5533, -0.4195],
         [-1.3368,  1.4415,  1.5294,  ...,  0.0772,  0.3692,  1.8368],
         [ 1.2540, -0.4037,  0.5048,  ..., -1.6621,  0.4997,  1.2072],
         ...,
         [ 0.5516,  0.1250,  0.4760,  ..., -2.7352,  0.0698,  0.5675],
         [ 0.8616, -0.3850,  0.2110,  ...,  1.0881, -1.3040, -0.1969],
         [ 0.5466, -0.3358, -1.4970,  ...,  0.3874,  1.1137, -0.6947]],

        [[-2.7485,  1.5554, -1.6776,  ..., -1.5061,  0.3527,  0.4425],
         [-0.3577,  0.0664,  1.6477,  ...,  1.4981,  0.8788,  0.7180],
         [-1.2437,  1.1667,  0.2717,  ..., -1.6202,  0.3641,  0.5971],
         ...,
         [ 0.5300, -0.2415,  0.8320,  ...,  0.1801,  1.2161, -0.7670],
         [ 0.0745, -0.1399, -0.0344,  ..., -0.4166, -0.1042,  0.5238],
         [ 0.8010, -1.6862,  0.5921,  ...,  0.2002, -2.0221,  0.0821]],

        [[-1.3214, -0.1141,  0.5650,  ...,  0.8438, -0.3967, -1.4232],
         [ 1.0757, -0.6970,  0.9134,  